In [1]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import *

from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.ml.feature import HashingTF, Normalizer
from pyspark.sql.types import StringType, ArrayType, DoubleType, IntegerType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
226,application_1575505342345_0271,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


#### Change parameters

In [2]:
COUNTRY = 'MX'
SALES_CAMP_FROM = '201818'
SALES_CAMP_TO = '202001'
ANIOCAMPANA = '202005'

## Reading dproducto table

In [3]:
s3_loc = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dproducto/'
df_dproducto = spark.read.parquet(s3_loc)\
                              .drop('cuc','codcategoria','codsubcategoria','codtipo','codsubtipo') \
                              .filter(col('codcuc') != '').cache() \
                              .withColumnRenamed('codcategoria_homo','codcategoria')\
                              .withColumnRenamed('codsubcategoria_homo', 'codsubcategoria')\
                              .withColumnRenamed('codtipo_homo', 'codtipo')\
                              .withColumnRenamed('codsubtipo_homo', 'codsubtipo')

In [4]:
df_dproducto.write.mode('overwrite')\
   .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/df_dproducto/")

## Reading Dtipooferta table

In [5]:
s3_loc1 = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dtipooferta/'
df_dtipooferta = spark.read.parquet(s3_loc1).filter(col('codpais')==COUNTRY)

In [6]:
df_dtipooferta.write.mode('overwrite')\
  .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/df_dtipooferta/")

## Reading sales data

In [7]:
s3_loc2 = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_fvtaproebecam/'
df_x = spark.read.parquet(s3_loc2).filter((col('aniocampana') == col('aniocampanaref')) & (col('aniocampana') >= SALES_CAMP_FROM)\
                              & (col('aniocampana') <= SALES_CAMP_TO) & (col('codpais')==COUNTRY)\
                              ).join(df_dtipooferta.filter(col('CodTipoProfit') == '01'), ['codpais', 'codtipooferta'],'inner'\
                                    ).select('codebelista', 'nrofactura', 'aniocampana', 'aniocampanaref', 'codtipooferta', \
                                             'codpais', 'codsap')

In [8]:
df_x.write.mode('overwrite')\
   .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/df_x/")

## Reading dmatrizcampana

In [9]:
s3_loc3 = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dmatrizcampana/'
dwh_dmatrizcampana = spark.read.parquet(s3_loc3)\
    .filter(col('codpais') == COUNTRY)\
    .filter(col('aniocampana') == ANIOCAMPANA)\
    .filter(col('codventa') != '00000')\
    .join(df_dtipooferta.filter(col('CodTipoProfit') == '01').select('codtipooferta').distinct(), 
          ['codtipooferta'], 'inner')\
    .withColumn('codtipooferta', col('codtipooferta').cast(IntegerType()))\
    .filter(col('codtipooferta')>= 200)\
    .filter(((col('vehiculoventa')!='CATALOGO') & ((col('vehiculoventa')!='REVISTA'))) | col('vehiculoventa').isNull())\
    .select('codestrategia','codventa','numoferta','preciooferta','codsap',
            'factorrepeticion', 'factorcuadre', 'codpais','aniocampana','vehiculoventa', 'codtipooferta')\
    .withColumn('code_numo',concat(col('codestrategia'), lit('_'), col('numoferta')))


In [10]:
#dwh_dmatrizcampana2 = dwh_dmatrizcampana.where(col("aniocampana").isin({ANIOCAMPANA}))#\
                    #    .where(col("codventa").isin({"78365"}))
#dwh_dmatrizcampana.show()

In [11]:
dwh_dmatrizcampana8 = spark.read.parquet(s3_loc3)\
   .filter(col('codpais') == COUNTRY)\
 .filter(col('codventa') != '00000')\
  .filter(col('aniocampana') == ANIOCAMPANA)\
 .filter(col('codtipooferta')>= 200)
dwh_dmatrizcampana8.show(50)

+-------------+-----------+-------------+-------------+--------+-----------------+--------------------+---------+---------+--------------+------------+--------------------+---------------+-----------+-------------+----------+---------------+--------------------+----------+------------+----------+--------------+----------------+-------------+---------+-----------------+------------------+-------------+-------------------+------------+----------------+------------+------------------+----------------+---------+--------+-------------------+------------------+-------------------+------------------+-------------+--------+----------------+----------+--------+-------+-----------+
|codcanalventa|codcatalogo|codestrategia|codtipooferta|codventa|      descatalogo|       destipooferta|nropagina|numoferta|precionormalmn|preciooferta|preciovtapropuestomn|codtipocatalogo|desargventa|desexposicion|desladopag|destipocatalogo|desubicacioncatalogo|fotomodelo|fotoproducto|nropaginas|paginacatalogo|desobse

In [12]:
#dwh_dmatrizcampana3 = spark.read.parquet(s3_loc3)
#dwh_dmatrizcampana3 = dwh_dmatrizcampana3.where(col("aniocampana").isin({ANIOCAMPANA}))\
                 #       .where(col("codventa").isin({"83799"}))
#dwh_dmatrizcampana3.show()

In [13]:
dwh_dmatrizcampana.write.mode('overwrite')\
.parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/dwh_dmatrizcampana/")

## Reading Offer details

In [14]:
offers_file = spark.read.parquet('s3://belc-bigdata-landing-dlk-qas/lan-ganamas/input/offer/country=MX/campaign=202005///')\
                .dropDuplicates()\
                .orderBy(col('Padre_Product_flag').desc())\
                .groupBy(['offer_id', 'cuc', 'palanca', 'units', 'offer_price_per_unit'])\
                .agg(first(col('Padre_Product_flag'),ignorenulls=True).alias('padre_flag'))\
                .withColumn('offer_id', col('offer_id').cast('string'))\
                .withColumnRenamed('offer_id','CUV')

#cambio de tipo de dato

## Reading Offer x Consultant 

In [15]:
cns_offer_file = spark.read.parquet('s3://belc-bigdata-landing-dlk-qas/lan-ganamas/input/cons/country=MX/campaign=202005//')\
                .withColumn('codigoconsultora', col('codigoconsultora').cast('string'))\
                .withColumnRenamed('codigoconsultora', 'codebelista')\
                .withColumn('CUV', col('CUV').cast('string'))\
                .dropDuplicates()\
                .drop('country','campaniaid')
###Implement this filter in daily_run script on offer_level_data table.
# \
#                 .join(cart_cns, ['codebelista'], 'inner')



cns_offer_detail = cns_offer_file.join(offers_file, ['CUV', 'palanca'], 'inner')

cns_offer_data = cns_offer_detail.withColumnRenamed('CUV', 'codventa')\
                            .withColumnRenamed('cuc', 'codcuc')\
                            .withColumnRenamed('offer_price_per_unit', 'preciooferta')\
                            .withColumnRenamed('units', 'factorrepeticion')\
                            .select('codebelista', 'codventa', 'codcuc', 
                                    'preciooferta', 'factorrepeticion', 'padre_flag')


target_data_rhs = cns_offer_data.withColumn('cuc_tot_price', col('factorrepeticion') * (col('preciooferta')))\
                    .drop('factorrepeticion', 'preciooferta')

In [16]:
cns_offer_detail.show(5)

+------+-------+-----------+-----------+-----+--------------------+----------+
|   CUV|palanca|codebelista|        cuc|units|offer_price_per_unit|padre_flag|
+------+-------+-----------+-----------+-----+--------------------+----------+
|312617|    OPT|    1955163|P0197089001|    1|                  62|         1|
|264421|    OPT|    1965468|  200083139|    3|                 306|         1|
|100949|     SR|    2045005|  200096670|    1|                 130|         1|
|264180|    OPT|    1955313|  200058729|    1|                 184|         1|
|100452|     SR|    0183013|  200083596|    2|                 231|         1|
+------+-------+-----------+-----------+-----+--------------------+----------+
only showing top 5 rows

In [17]:
offers_file.show()

+------+-----------+-------+-----+--------------------+----------+
|   CUV|        cuc|palanca|units|offer_price_per_unit|padre_flag|
+------+-----------+-------+-----+--------------------+----------+
|100000|  200004616|     SR|    1|                 286|         1|
|100112|  200052058|     SR|    1|                 106|         1|
|100213|  200060404|     SR|    4|                 178|         1|
|100286|  200069897|     SR|    5|                  68|         1|
|100297|  200070075|     SR|    7|                  50|         1|
|100314|  200074678|     SR|    2|                  66|         1|
|100509|  200087327|     SR|    3|                  51|         1|
|100812|  200094901|     SR|   10|                  75|         1|
|100986|  200097895|     SR|    4|                  31|         1|
|101244|  200054026|     SR|    3|                  35|         0|
|101342|  200051086|     SR|    3|                  84|         0|
|101778|P0197089001|     SR|    1|                  61|       

In [18]:
cns_offer_file\
   .filter(col('palanca') == 'OPT)

EOL while scanning string literal (<stdin>, line 2)
  File "<stdin>", line 2
    .filter(col('palanca') == 'OPT)
                                  ^
SyntaxError: EOL while scanning string literal



In [19]:
offers_file2 = offers_file\
   .filter(col('palanca') == 'SR')
offers_file2.show()


+------+-----------+-------+-----+--------------------+----------+
|   CUV|        cuc|palanca|units|offer_price_per_unit|padre_flag|
+------+-----------+-------+-----+--------------------+----------+
|100000|  200004616|     SR|    1|                 286|         1|
|100112|  200052058|     SR|    1|                 106|         1|
|100213|  200060404|     SR|    4|                 178|         1|
|100286|  200069897|     SR|    5|                  68|         1|
|100297|  200070075|     SR|    7|                  50|         1|
|100314|  200074678|     SR|    2|                  66|         1|
|100509|  200087327|     SR|    3|                  51|         1|
|100812|  200094901|     SR|   10|                  75|         1|
|100986|  200097895|     SR|    4|                  31|         1|
|101244|  200054026|     SR|    3|                  35|         0|
|101342|  200051086|     SR|    3|                  84|         0|
|101778|P0197089001|     SR|    1|                  61|       

In [20]:
cns_offer_file_valid= cns_offer_file.where(col("palanca").isin({"OPT"}))
cns_offer_file_valid.show()

+-------+-----------+------+
|palanca|codebelista|   CUV|
+-------+-----------+------+
|    OPT|    1354137|264505|
|    OPT|    1363043|264944|
|    OPT|    1410227|264223|
|    OPT|    1412017|312341|
|    OPT|    1500763|312149|
|    OPT|    1424997|312758|
|    OPT|    1615955|311921|
|    OPT|    1500763|313181|
|    OPT|    1720763|264523|
|    OPT|    1748459|264699|
|    OPT|    1748459|264906|
|    OPT|    1758723|318538|
|    OPT|    1759831|264131|
|    OPT|    1801448|313355|
|    OPT|    1801448|314792|
|    OPT|    1761786|264454|
|    OPT|    1801448|316883|
|    OPT|    1807804|264512|
|    OPT|    1807441|264320|
|    OPT|    1826536|311881|
+-------+-----------+------+
only showing top 20 rows

In [21]:
#cns_offer_file.show(5)
cns_offer_file_valid= cns_offer_file.where(col("palanca").isin({"OPM"}))
cns_offer_file_valid.show()

+-------+-----------+---+
|palanca|codebelista|CUV|
+-------+-----------+---+
+-------+-----------+---+

In [22]:
#offers_file.show()
offers_file_valid= offers_file.where(col("palanca").isin({"SR"}))
offers_file_valid.show()

+------+-----------+-------+-----+--------------------+----------+
|   CUV|        cuc|palanca|units|offer_price_per_unit|padre_flag|
+------+-----------+-------+-----+--------------------+----------+
|100000|  200004616|     SR|    1|                 286|         1|
|100112|  200052058|     SR|    1|                 106|         1|
|100213|  200060404|     SR|    4|                 178|         1|
|100286|  200069897|     SR|    5|                  68|         1|
|100297|  200070075|     SR|    7|                  50|         1|
|100314|  200074678|     SR|    2|                  66|         1|
|100509|  200087327|     SR|    3|                  51|         1|
|100812|  200094901|     SR|   10|                  75|         1|
|100986|  200097895|     SR|    4|                  31|         1|
|101244|  200054026|     SR|    3|                  35|         0|
|101342|  200051086|     SR|    3|                  84|         0|
|101778|P0197089001|     SR|    1|                  61|       

In [23]:
target_data_rhs.write.mode('overwrite')\
    .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/target_data_rhs/")

In [24]:
cns_offer_file2 = cns_offer_file\
   .filter(col('palanca') == 'OPT')
cns_offer_file2.show()


+-------+-----------+------+
|palanca|codebelista|   CUV|
+-------+-----------+------+
|    OPT|    1354137|264505|
|    OPT|    1363043|264944|
|    OPT|    1410227|264223|
|    OPT|    1412017|312341|
|    OPT|    1500763|312149|
|    OPT|    1424997|312758|
|    OPT|    1615955|311921|
|    OPT|    1500763|313181|
|    OPT|    1720763|264523|
|    OPT|    1748459|264699|
|    OPT|    1748459|264906|
|    OPT|    1758723|318538|
|    OPT|    1759831|264131|
|    OPT|    1801448|313355|
|    OPT|    1801448|314792|
|    OPT|    1761786|264454|
|    OPT|    1801448|316883|
|    OPT|    1807804|264512|
|    OPT|    1807441|264320|
|    OPT|    1826536|311881|
+-------+-----------+------+
only showing top 20 rows

## Calculating the offer_price for cns_offer_file (RHS)

In [25]:
cns_offer_cuc_data = target_data_rhs.join(df_dproducto,['codcuc'],'left')\
                                        .select('codebelista',
                                               'codventa','codcuc','desmarca','dessubtipo', 'padre_flag', 'cuc_tot_price')\
                                        .dropDuplicates()\
                                        .withColumn('brand_description',concat(col('desmarca'), lit(' '), col('dessubtipo')))\
                                        .drop('desmarca','dessubtipo')\
                                        .groupBy(['codebelista', 'codventa', 'codcuc'])\
                                        .agg(first(col('padre_flag')).alias('padre_flag'), 
                                             first(col('cuc_tot_price')).alias('cuc_tot_price'),
                                             concat_ws(' ', collect_list(col('brand_description'))).alias('brand_description')
                                            )

### calculating offer price and creating the text column 


###Below line of code can be implemented in daily run script, instead of saving this file also.
##Read from cns_offer_cuc_data file and then create offer_level_data.
# offer_level_data = cns_offer_cuc_data.withColumn('padre_cuc',when(col('padre_flag')==1,col('codcuc')))\
#                                .groupBy(['codventa','codebelista'])\
#                                .agg(sum(col('cuc_tot_price')).alias('offer_price'),
#                                     concat_ws(' ', collect_list(col('brand_description'))).alias('text'),
#                                     first(col('padre_cuc'),ignorenulls=True).alias('padre_cuc')) \
#                                .withColumn("text", split(col('text'), ' '))

In [26]:
###NOTE: when you read these two files file in daily script, 
##remember to filter out only those cns whose data is present in cart
##.join(cart_cns, ['codebelista'], 'inner')
cns_offer_cuc_data.write.mode('overwrite')\
.parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/cns_offer_cuc_data/")

# offer_level_data.write.mode('overwrite').parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/offer_level_data/")

## Cross Sell Calculation

In [27]:
def calculate_support_and_lift(df):
    min_support = 0.001
    min_confidence = 0
    min_lift = 1.05
    #if(info[country]=='PE' or info[country]=='CO' ):
        #min_support = 0.01
    fpg = FPGrowth(itemsCol='items', minSupport=min_support, minConfidence=min_confidence)
    model = fpg.fit(df)
    # creating one side mapping of frequency.
    freq_itemsets = model.freqItemsets
    df_tuple = freq_itemsets.filter(size(col('items')) == 2)
    df_base = df_tuple.withColumn('x',col('items')[0]).withColumn('y',col('items')[1]).drop('items') \
                      .withColumnRenamed('freq', 'freq_xy')
    df_individual = freq_itemsets.filter(size(col('items')) == 1).withColumn('items',col('items')[0])
    # support = # T(c1 & c2) /Total
    # lift = (# T(c1 & c2) * total) /(T(c2)*T(c1))
    # Removing multiplication or division of total as normalization step is already present.
    df_con_lift_support = df_base.join(df_individual.withColumnRenamed('freq', 'freq_x').withColumnRenamed('items','x'),['x'],'left') \
                           .join(df_individual.withColumnRenamed('freq', 'freq_y').withColumnRenamed('items','y'),['y'],'left') \
                           .withColumn('support', col('freq_xy')) \
                           .withColumn('lift', col('freq_xy')/(col('freq_x'))/(col('freq_y')))
    min_max = df_con_lift_support.agg(min(col('support')).alias('min_support'),
                                      max(col('support')).alias('max_support'),
                                      min(col('lift')).alias('min_lift'),
                                      max(col('lift')).alias('max_lift'))
    normalized_lift_support = df_con_lift_support.crossJoin(broadcast(min_max))\
                                                 .withColumn('norm_support',(col('support')-col('min_support'))/(col('max_support')-col('min_support')) ) \
                                                 .withColumn('norm_lift', (col('lift')-col('min_lift'))/(col('max_lift')-col('min_lift')))\
                                                 .withColumn('score',col('norm_support')+col('norm_lift'))
    # returning both side mapping.
    return normalized_lift_support.select('x','y','score').union(normalized_lift_support.select('y','x','score'))

## Calculating CUC score

In [28]:
##NOTE: THIS CALCULATION HAS BEEN MOVED TO DAILY RUN SCRIPT

# df_p = df_x.join(df_dproducto, ['codsap'], 'inner').select('codebelista', 'nrofactura', 'codcuc').dropDuplicates()
# #df_offers_con = cart_offer_cuc_data.withColumnRenamed('cuc', 'codcuc').drop('codebelista')
# df_offers_con = target_data_lhs.withColumnRenamed('cuc', 'codcuc').drop('codebelista')
# df_m = df_p.join(df_offers_con, ['codcuc'], 'inner').select('codebelista', 'nrofactura', 'codcuc').dropDuplicates()
# cuc_data = df_m.withColumn('unique_tr_id',concat(col('codebelista'),col('nrofactura')))
# cuc_base_data = cuc_data.select('unique_tr_id', 'codcuc').dropDuplicates() \
#                              .groupBy('unique_tr_id').agg(collect_list(col('codcuc')).alias('items'))

# cuc_score = calculate_support_and_lift(cuc_base_data).withColumnRenamed('x','codcuc')\
#                             .withColumnRenamed('y','codcuc_1')\
#                             .withColumnRenamed('score','cuc_score')


In [29]:
# cuc_score.write.mode('overwrite')\
#     .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/cuc_score/")

## Calculating Brand_Cat score

In [30]:
df_p = df_x.join(df_dproducto, ['codsap'], 'inner').select('codebelista', 'nrofactura', 'codcuc', 
                                                           'desmarca', 'descategoria').dropDuplicates()
cuc_data = df_p.withColumn('unique_tr_id',concat(col('codebelista'),col('nrofactura')))
brand_cat_base_data = cuc_data.select('unique_tr_id', 'desmarca','descategoria').dropDuplicates() \
                                   .withColumn('brand-category',concat(col('desmarca'),col('descategoria')))\
                                   .groupBy('unique_tr_id').agg(collect_list(col('brand-category')).alias('items'))
brand_cat_score = calculate_support_and_lift(brand_cat_base_data).withColumnRenamed('x','brand_category')\
                        .withColumnRenamed('y','brand_category_1')\
                        .withColumnRenamed('score','brand_category_score')

In [31]:
brand_cat_base_data.show()


+-----------------+--------------------+
|     unique_tr_id|               items|
+-----------------+--------------------+
|00009091902070041|[LBELPROMOCION US...|
|00016841905815984|[ESIKAMUESTRAS CO...|
|00050931901680535|[LBELMAQUILLAJE, ...|
|00055681903831978|[CYZONEFRAGANCIAS...|
|00059081903320846|[LBELFRAGANCIAS, ...|
|00120301903329424|[LBELFRAGANCIAS, ...|
|00139181900977583|[ESIKATRATAMIENTO...|
|00142221901258681|[ESIKAMAQUILLAJE,...|
|00157901904356777|[ESIKAFRAGANCIAS,...|
|00168331904147050|[CYZONEMAQUILLAJE...|
|00180621901291664|[ESIKADAMAS, LBEL...|
|00194731902034017|[ESIKAFRAGANCIAS,...|
|00201831900169960|[LBELTRATAMIENTO ...|
|00210611901004298|[ESIKAMAQUILLAJE,...|
|00215811901935017|[LBELTRATAMIENTO ...|
|00222511805831465|[LBELPROMOCION US...|
|00230861903432227|[LBELTRATAMIENTO ...|
|00237741903809160|[CYZONEHOGAR, LBE...|
|00255951902794272|[LBELPROMOCION US...|
|00258551900561299|[CYZONECUIDADO PE...|
+-----------------+--------------------+
only showing top

In [32]:
brand_cat_score.write.mode('overwrite')\
    .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/brand_cat_score/")

## Calculating Brand_Subcat score

In [33]:
df_p = df_x.join(df_dproducto, ['codsap'], 'inner').select('codebelista', 'nrofactura', 'codcuc', 
                                                           'desmarca', 'dessubcategoria').dropDuplicates()
cuc_data = df_p.withColumn('unique_tr_id',concat(col('codebelista'),col('nrofactura')))
brand_subcat_base_data = cuc_data.select('unique_tr_id', 'desmarca','dessubcategoria').dropDuplicates() \
                                   .withColumn('brand-subcategory',concat(col('desmarca'),col('dessubcategoria')))\
                                   .groupBy('unique_tr_id').agg(collect_list(col('brand-subcategory')).alias('items'))
brand_subcat_score = calculate_support_and_lift(brand_subcat_base_data).withColumnRenamed('x','brand_subcategory')\
                                                .withColumnRenamed('y','brand_subcategory_1')\
                                                .withColumnRenamed('score','brand_subcategory_score')


In [34]:
brand_subcat_score.write.mode('overwrite')\
    .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/brand_subcat_score/")


## Calculating Brand_Subtype score

In [35]:
df_p = df_x.join(df_dproducto, ['codsap'], 'inner').select('codebelista', 'nrofactura', 'codcuc', 'desmarca', 'dessubtipo').dropDuplicates()
cuc_data = df_p.withColumn('unique_tr_id',concat(col('codebelista'),col('nrofactura')))
brand_subtype_base_data = cuc_data.select('unique_tr_id', 'desmarca','dessubtipo').dropDuplicates() \
                                   .withColumn('brand-subtype',concat(col('desmarca'),col('dessubtipo')))\
                                   .groupBy('unique_tr_id').agg(collect_list(col('brand-subtype')).alias('items'))
brand_subtype_score = calculate_support_and_lift(brand_subtype_base_data).withColumnRenamed('x','brand_subtype').withColumnRenamed('y','brand_subtype_1').withColumnRenamed('score','brand_subtype_score')

In [43]:
brand_subtype_base_data.show()

+-----------------+--------------------+
|     unique_tr_id|               items|
+-----------------+--------------------+
|00009091902070041|[LBELPROMOCION US...|
|00016841905815984|[CYZONEMAQUILLAJE...|
|00050931901680535|[LBELTRATAMIENTO ...|
|00055681903831978|[CYZONEMAQUILLAJE...|
|00059081903320846|[CYZONECOMPLEMENT...|
|00120301903329424|[LBELFRAGANCIAS/D...|
|00139181900977583|[ESIKAMAQUILLAJE/...|
|00142221901258681|[ESIKAMAQUILLAJE/...|
|00157901904356777|[CYZONEACCESORIOS...|
|00168331904147050|[ESIKAFRAGANCIAS/...|
|00180621901291664|[CYZONETRATAMIENT...|
|00194731902034017|[LBELMAQUILLAJE/P...|
|00201831900169960|[ESIKAFRAGANCIAS/...|
|00210611901004298|[ESIKATRATAMIENTO...|
|00215811901935017|[LBELPROMOCION US...|
|00222511805831465|[LBELCUIDADO PERS...|
|00230861903432227|[LBELMAQUILLAJE/P...|
|00237741903809160|[CYZONEMAQUILLAJE...|
|00255951902794272|[CYZONECOMPLEMENT...|
|00258551900561299|[CYZONEFRAGANCIAS...|
+-----------------+--------------------+
only showing top

In [44]:
df_p.show()

+-----------+----------+-----------+--------+--------------------+
|codebelista|nrofactura|     codcuc|desmarca|             destipo|
+-----------+----------+-----------+--------+--------------------+
|    1945876|1902565158|P0280105005|    LBEL|MAQUILLAJE/PARA R...|
|    1845652|1902279239|P0261051003|    LBEL|MAQUILLAJE/PARA R...|
|    1495191|1902233237|P0210060001|   ESIKA|MAQUILLAJE/PARA O...|
|    0366017|1902525372|  200042358|    LBEL|FRAGANCIAS/CABALL...|
|    1943397|1902642585|  200088336|   ESIKA|FRAGANCIAS/CABALL...|
|    1740883|1902528155|  210091710|    LBEL|PROMOCION USUARIO...|
|    1903769|1902529331|  200095931|   ESIKA|FRAGANCIAS/DAMAS/...|
|    1888172|1902645827|P0210085000|   ESIKA|MAQUILLAJE/PARA L...|
|    0457151|1902523982|P0292137005|   ESIKA|MAQUILLAJE/PARA L...|
|    1925600|1902526219|  200011767|   ESIKA|FRAGANCIAS/UNISEX...|
|    1844937|1902354641|  200004616|    LBEL|TRATAMIENTO FACIA...|
|    1182153|1902526067|P0261059004|    LBEL|MAQUILLAJE/PARA O

In [45]:
cuc_data.show()

+-----------+----------+-----------+--------+--------------------+-----------------+
|codebelista|nrofactura|     codcuc|desmarca|             destipo|     unique_tr_id|
+-----------+----------+-----------+--------+--------------------+-----------------+
|    0083346|1901632604|  210090573|   ESIKA|BIJOUTERIE/DAMAS/...|00833461901632604|
|    0083610|1901786407|P0292082007|   ESIKA|MAQUILLAJE/PARA O...|00836101901786407|
|    0084278|1901741444|P0280078000|    LBEL|MAQUILLAJE/PARA R...|00842781901741444|
|    0085671|1901937563|P0292090005|   ESIKA|MAQUILLAJE/PARA O...|00856711901937563|
|    0087148|1901704691|  200059602|    LBEL|TRATAMIENTO FACIA...|00871481901704691|
|    0049840|1901594774|  200091463|   ESIKA|FRAGANCIAS/DAMAS/...|00498401901594774|
|    0061137|1901869860|P0292141009|   ESIKA|MAQUILLAJE/PARA O...|00611371901869860|
|    0060825|1901633144|P0194067000|  CYZONE|MAQUILLAJE/PARA O...|00608251901633144|
|    0056639|1901734486|  200011767|   ESIKA|FRAGANCIAS/UNISEX...

In [46]:
brand_subtype_score.show()

An error occurred while calling o856.showString.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:150)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.joins.BroadcastNestedLoopJ

In [37]:
brand_subtype_score.write.mode('overwrite')\
    .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/brand_subtype_score/")

An error occurred while calling o925.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at

## Calculating Brand_Type score

In [38]:
df_p = df_x.join(df_dproducto, ['codsap'], 'inner').select('codebelista', 'nrofactura', 'codcuc', 'desmarca', 'destipo').dropDuplicates()
cuc_data = df_p.withColumn('unique_tr_id',concat(col('codebelista'),col('nrofactura')))
brand_type_base_data = cuc_data.select('unique_tr_id', 'desmarca','destipo').dropDuplicates() \
                                   .withColumn('brand-type',concat(col('desmarca'),col('destipo')))\
                                   .groupBy('unique_tr_id').agg(collect_list(col('brand-type')).alias('items'))
brand_type_score = calculate_support_and_lift(brand_type_base_data).withColumnRenamed('x','brand_type').withColumnRenamed('y','brand_type_1').withColumnRenamed('score','brand_type_score')

In [39]:
brand_type_score.write.mode('overwrite')\
    .parquet("s3://belc-bigdata-landing-dlk-qas/lan-ganamas/inter_input/brand_type_score/")

An error occurred while calling o1127.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	a

In [40]:
#DF = spark.read.parquet('s3a://belc-bigdata-landing-dlk-qas/lan-ganamas/input/cart/year=2020/month=03/day=02/country=MX/')

In [41]:
#DF.count()